In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
from selenium.common.exceptions import TimeoutException

# This script has been executed once only to extract the brand names and models.

In [2]:
root_url = "https://www.autovit.ro"

In [ ]:
def extract_brands():
    options = Options()
    #options.add_argument("--headless=new")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(root_url)
    wait = WebDriverWait(driver, 10)

    try:
        consent_btn = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept')]"))
        )
        consent_btn.click()
        print("Consent popup closed")
    except:
        print("No consent popup found")

    brand_dropdown = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[aria-label="Marca"]'))
    )
    brand_dropdown.click()

    brand_elements = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[type="selectable"][aria-label]'))
    )

    brands = [el.get_attribute("aria-label").split(" (")[0] for el in brand_elements]

    brand_model_dict = {}

    for brand in brands[1:]:
        #Re-open dropdown each time (sometimes it closes after selection)
        brand_dropdown.click()
        brand_option = wait.until(
            EC.element_to_be_clickable((By.XPATH, f'//div[@type="selectable" and contains(@aria-label, "{brand}")]'))
        )
        brand_option.click()
        print(f"Selected brand: {brand}")

        #Wait for the model dropdown to become clickable
        model_dropdown = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[aria-label="Model"]'))
        )
        model_dropdown.click()
        time.sleep(1)

        #Wait for model options to appear
        try:
            model_elements = wait.until(
                EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'div[type="selectable"][role="checkbox"]'))
            )
            models = [el.get_attribute("aria-label").split(" (")[0] for el in model_elements]
        except TimeoutException:
            models = []
            print(f"No models found for brand: {brand}")
            continue

        brand_model_dict[brand] = models

    driver.quit()
    return brand_model_dict


In [32]:
brands = extract_brands()

Consent popup closed
Selected brand: BMW
Selected brand: Mercedes-Benz
Selected brand: Volkswagen
Selected brand: Audi
Selected brand: Ford
Selected brand: Renault
Selected brand: Skoda
Selected brand: Dacia
Selected brand: Peugeot
Selected brand: Opel
Selected brand: Abarth
Selected brand: Acura
Selected brand: Aiways
Selected brand: Aixam
Selected brand: Alfa Romeo
Selected brand: Allview
Selected brand: Altul
Selected brand: Aro
Selected brand: Aston Martin
Selected brand: Audi
Selected brand: Austin
Selected brand: Baic
Selected brand: Bentley
Selected brand: BMW
Selected brand: Bronco Sport
No models found for brand: Bronco Sport
Selected brand: Bugatti
Selected brand: Buick
Selected brand: Byd
Selected brand: Cadillac
Selected brand: Cenntro
Selected brand: Chatenet
Selected brand: Chery
Selected brand: Chevrolet
Selected brand: Chrysler
Selected brand: Citroën
Selected brand: Comarth
Selected brand: Cupra
Selected brand: Dacia
Selected brand: Daewoo
Selected brand: Daihatsu
Sele

In [37]:
for brand, models in brands.items():
    if "Selectează toate" in models:
        models.remove("Selectează toate")
    elif "Altul" in models:
        models.remove("Altul")
    elif "altul" in models:
        models.remove("altul")

In [38]:
with open("autovit_brands.json", "w", encoding="utf-8") as f:
    json.dump(brands, f, ensure_ascii=False, indent=2)